In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = "dirty_cafe_sales.csv"
cafe_report= pd.read_csv(data)
cafe_report.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [4]:
def clean_dataframe(df, numeric_columns=None, date_columns=None, cat_fill_value="Unknown", row_na_thresh=0.5):
    df = df.copy()

    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

    if date_columns:
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    if numeric_columns:
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df[col] = df[col].fillna(df[col].median())

    for col in df.select_dtypes(include=['object', 'category']).columns:
        df[col] = df[col].fillna(cat_fill_value)

    # modifiction if neccessary 
    """ 
    binary_like_cols = ['Discount Applied']  #customize this list
    for col in binary_like_cols:
        if col in df.columns:
            df[col] = df[col].map({1: 'Yes', 0: 'No', np.nan: 'Not Recorded'})
    """
    df = df.dropna(thresh=int(len(df.columns) * (1 - row_na_thresh)))

    return df


In [5]:
cafe_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [6]:
cleaned = clean_dataframe(cafe_report, numeric_columns=["Quantity","Price Per Unit","Total Spent"] ,date_columns=["Transaction Date"] )

In [7]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              10000 non-null  object        
 2   Quantity          10000 non-null  float64       
 3   Price Per Unit    10000 non-null  float64       
 4   Total Spent       10000 non-null  float64       
 5   Payment Method    10000 non-null  object        
 6   Location          10000 non-null  object        
 7   Transaction Date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


In [8]:
cleaned.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,8.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [ ]:
cleaned.dropna(subset=['Transaction Date'], inplace=True)
cleaned.iloc[11] #11 was NaT previously


Transaction ID              TXN_7619095
Item                           Sandwich
Quantity                            2.0
Price Per Unit                      4.0
Total Spent                         8.0
Payment Method                     Cash
Location                       In-store
Transaction Date    2023-05-03 00:00:00
Name: 12, dtype: object

In [10]:
cleaned

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,8.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Unknown,UNKNOWN,2023-08-30
9996,TXN_9659401,Unknown,3.0,3.0,3.0,Digital Wallet,Unknown,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,Unknown,2023-03-02
9998,TXN_7695629,Cookie,3.0,3.0,3.0,Digital Wallet,Unknown,2023-12-02


In [11]:
cat_cols = cleaned.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    cleaned[col] = cleaned[col].replace(['error', 'ERROR', 'Error', 'UNKNOWN', 'unknown', 'Unknown'], 'Unknown')
cleaned


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,8.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Unknown,Unknown,2023-08-30
9996,TXN_9659401,Unknown,3.0,3.0,3.0,Digital Wallet,Unknown,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,Unknown,2023-03-02
9998,TXN_7695629,Cookie,3.0,3.0,3.0,Digital Wallet,Unknown,2023-12-02


In [ ]:
def summarize_df(df):
    print("Shape:", df.shape)
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nData Types:\n", df.dtypes)
    print("\nSample Unique Value Counts:")
    for col in df.columns:
        print(f"\n{col}:\n", df[col].value_counts(dropna=False).head())

summarize_df(cleaned)

Shape: (9540, 8)

Missing Values:
 Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

Data Types:
 Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object

Sample Unique Value Counts:

Transaction ID:
 Transaction ID
TXN_1961373    1
TXN_3894173    1
TXN_7436097    1
TXN_4307602    1
TXN_2394502    1
Name: count, dtype: int64

Item:
 Item
Juice       1124
Coffee      1123
Salad       1099
Cake        1082
Sandwich    1075
Name: count, dtype: int64

Quantity:
 Quantity
3.0    2220
5.0    1914
2.0    1892
4.0    1774
1.0    1740
Name: count, dtype: int64

Price Per Unit:
 Price Per Unit
3.0    2825
4.0    2219
2.0    